# TextToSQL

- Author: [Jaehun Choi](https://github.com/ash-hun)
- Design: 
- Peer Review: 
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb)

## Overview

이 튜토리얼은 임의의 데이터베이스 정보를 받아와 SQL Query문을 생성하는 기능을 제공합니다. 해당 튜토리얼에서 데이터베이스 정보를 추출하는 부분은 다루지 않으나 특정 테이블의 칼럼정보 및 자세한 설명이 해당할 수 있으며, OpenAI의 GPT 모델(예: gpt-4o)과 프롬프트 템플릿을 사용하여 SQL 쿼리문을 생성하게 됩니다.

특징
- 데이터베이스 정보 : SQL 쿼리문 생성을 위한 데이터베이스 정보 형태에 대해 소개합니다.
- TextToSQL : 데이터베이스의 정보를 바탕으로 맞춤 SQL 쿼리문을 생성합니다.

### Table of Contents

- [Overview](#overview)
- [Environement Setup](#environment-setup)
- [Database Information](#database-information)
- [TextToSQL](#text-to-sql)


### References

- [JsonOutputParser](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.json.JsonOutputParser.html)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "01-TextToSQL",  # title 과 동일하게 설정해 주세요
    }
)

Environment variables have been set successfully.


You can alternatively set API keys such as `OPENAI_API_KEY` in a `.env` file and load them.

[Note] This is not necessary if you've already set the required API keys in previous steps.

In [4]:
# Load API keys from .env file
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Database Information

데이터베이스 정보에 대한 설명은 크게 2가지 형태를 활용할 수 있습니다.  

- 테이블 혹은 칼럼정보를 가공없이 주입하는 경우  
- 테이블 혹은 칼럼정보를 description화하여 주입하는 경우  


본 튜토리얼에서는 아래의 예제 테이블을 바탕으로 진행하고자 합니다.

- 데이터베이스 이름: **CompanyDB**
- employees (직원 정보) 테이블의 칼럼정보

```
    id (INT, PRIMARY KEY, AUTO_INCREMENT)
    name (VARCHAR, 직원 이름)
    position (VARCHAR, 직책)
    department (VARCHAR, 부서명)
    salary (DECIMAL, 연봉)
    hire_date (DATE, 입사일)
    departments (부서 정보)
```


In [5]:
# 테이블 혹은 칼럼정보를 가공없이 주입하는 경우

db_info_directly = """
employees table
- id (INT, PRIMARY KEY, AUTO_INCREMENT)
- name (VARCHAR, 직원 이름)
- position (VARCHAR, 직책)
- department (VARCHAR, 부서명)
- salary (DECIMAL, 연봉)
- hire_date (DATE, 입사일)
- departments (부서 정보)
"""

In [6]:
# 테이블 혹은 칼럼정보를 description화하여 주입하는 경우

db_info_description = """
The employees table stores information about the employees in the organization. It includes the following fields:

- id: An integer that serves as the primary key and is auto-incremented for each employee.
- name: A string (VARCHAR) representing the name of the employee.
- position: A string (VARCHAR) indicating the job title or position of the employee.
- department: A string (VARCHAR) specifying the department to which the employee belongs.
- salary: A decimal value representing the employee's salary.
- hire_date: A date field indicating when the employee was hired.

"""

## Text to SQL

데이터베이스의 정보를 바탕으로 맞춤 SQL 쿼리문을 생성합니다.

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class SqlSchema(BaseModel):
    statement: str = Field(description="SQL Query Statement")

In [8]:
# Create an OpenAI object
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [9]:
# Write your question
question = "Please show me the names and job titles of all employees in the Engineering department."

# Set up the parser and inject the instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=SqlSchema)

In [10]:
# Set up the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a expert of SQL. Answer questions concisely."),
        ("user", "Please generate SQL Query statement from Schema_Info.\n\n#Format: {format_instructions}\n\n#Schema_Info: {schema_info}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# Combine the prompt, model, and JsonOutputParser into a chain
chain = prompt | model | parser

# Run the chain with your question : raw style prompt
answer = chain.invoke({"schema_info": db_info_directly, "question": question})
answer

{'statement': "SELECT name, position FROM employees WHERE department = 'Engineering';"}

In [11]:
# Run the chain with your question : description style prompt
answer = chain.invoke({"schema_info": db_info_description, "question": question})
answer

{'statement': "SELECT name, position FROM employees WHERE department = 'Engineering';"}